# Monte Carlo Methods III: Parameter Estimation and Markov Chain

This hands-on is based on [Gravitational Wave Open Data Workshop 2024](https://github.com/gw-odw/odw-2024/).

## Introduction

Gravitational waves are ripples in spacetime predicted by Albert Einstein's General Theory of Relativity.
These waves propagate outward from sources such as merging black holes, neutron star collisions, or rapidly rotating neutron stars.
The first direct detection of gravitational waves, GW150914, was observed by the LIGO detectors in 2015, opening a new window into observing astrophysical phenomena previously inaccessible by electromagnetic observations alone.

Detecting and analyzing gravitational waves allows astronomers and physicists to study objects and phenomena that emit very little or no light, providing valuable insights into the behavior of gravity in extreme environments and the properties of dense objects like black holes and neutron stars.

In this hands-pon, we introduce Bayesian inference methods essential for estimating the parameters of gravitational wave signals detected by instruments such as LIGO and Virgo.
Specifically, we'll demonstrate parameter estimation using both a rejection method and Monte Carlo Markov Chain.

## Bayesian Inference: A Brief Overview

Recalling that Bayesian inference allows us to update our knowledge of model parameters based on observed data.
Bayes' theorem mathematically represents this process:
\begin{align}
  p(\theta|d,M) = \frac{\mathcal{L}(d|\theta,M) \pi(\theta|M)}{\mathcal{Z}(d|M)}
\end{align}
where:
* $p(\theta|d, M)$ is the posterior distribution.
* $\mathcal{L}(d|\theta, M)$ is the likelihood function, measuring agreement between model predictions and observed data.
* $\pi(\theta|M)$ is the prior distribution, representing previous knowledge.
* $\mathcal{Z}(d|M)$ is the evidence, normalizing the posterior.

Typically, the posterior cannot be computed analytically, requiring computational approximations.
Stochastic (Monte Carlo) sampling is a common method for this purpose.

## Toy Model

We download sample observational data (`toy_model.csv`) containing simulated time-series observations:

In [ ]:
import os
if not os.path.isfile("toy_model.csv"):
    print("Downloading toy_model.csv")
    ! wget https://raw.githubusercontent.com/gw-odw/odw-2024/main/Tutorials/Day_3/toy_model.csv
else:
    print("toy_model.csv exists; not downloading")

Our example data (`toy_model.csv`) contains measurements (`yobs`) recorded at specific times (`time`).
Let's visualize these observations:

In [ ]:
import numpy as np

time, yobs = np.genfromtxt("toy_model.csv", delimiter=",").T

In [ ]:
from matplotlib import pyplot as plt

plt.plot(time, yobs)
plt.xlabel("Time [s]")
plt.ylabel("Observed values")
plt.title("Simulated Observational Data")

We propose the following sine-Gaussian model to explain the data:
\begin{align}
  s(t; f,\alpha) = e^{-(t/\alpha)^2} \sin(2\pi f t),
\end{align}
with frequency parameter $f$ and damping parameter $\alpha$.
To build intuition, we visualize this model for representative parameters:

In [ ]:
def sinusoid(time, freq):
    return np.sin(2 * np.pi * freq * time)

def gaussian_exponential(time, alpha):
    return np.exp(-(time/alpha)**2)

def model_Ms(time, freq, alpha):
    return gaussian_exponential(time, alpha) * sinusoid(time, freq)

In [ ]:
freq, alpha = 2, 0.5

plt.plot(time, sinusoid(time, freq), label="Sinusoid")
plt.plot(time, gaussian_exponential(time, alpha), label="Gaussian envelope")
plt.plot(time, model_Ms(time, freq, alpha), label="Sine-Gaussian")
plt.xlabel("Time [s]")
plt.ylabel("Model components")
plt.legend()

## Parameter Estimation

With the data (`yobs`) and our model $M_s$, we can now estimate the parameters $f$ and $\alpha$.
To do this, similar to previous lectures, we will use Bayes theorem, i.e. we want to approximate the distribution
\begin{align}
p(\theta | d, M_s) = \frac{\mathcal{L}(d| \theta, M_s) \;\pi(\theta | M_s)}{\mathcal{Z}(d | M_s)}
\end{align}
where $\theta=\{f, \alpha\}$ is the two-dimensional parameter vector and $d$ is `yobs` (measured at times `time`).

To this end, we need to define the likelihood and priors.
Note that, if we are **only interested in the shape of the distribution**, then we can ignore the evidence, i.e. we can estimate the unnormalized distribution
\begin{align}
  p(\theta | d, M_s) \propto \mathcal{L}(d| \theta, M_s) \;\pi(\theta | M_s)
\end{align}

### Likelihood

For this toy example, we will assume that the data consists of the generate model $M_s$ and additive white Gaussian noise, i.e.
\begin{align}
y_{\rm obs}(t) = s(t; f, \alpha) + \epsilon
\end{align}
where $\epsilon \sim N(0, \sigma)$ by which we mean that $\epsilon$ is drawn from a Gaussian distribution with zero mean and standard deviation $\sigma=0.1$ (for now, we will assume this is known a priori, but see challenges below for how it could be estimated). 

This definition of how the data was created allows us to define our likelihood.
Namely, given a value of $\{f, \alpha\}$, the likelihood of a single data point $y_i$ (measured at $t_i$) is:
\begin{align}
\mathcal{L}(y_i| f, \alpha, M_s) = \frac{1}{\sqrt{2\pi \sigma^2}} \exp\left(-\frac{(y_i - s(t_i; f, \alpha))^2}{2\sigma^2}\right)
\end{align}

To extend this to multiple data points, we assume they are independent then
\begin{align}
\mathcal{L}(y_{obs} | f, \alpha, M_s) = \prod_i \mathcal{L}(y_i| f, \alpha, M_s)
\end{align}
In practice, it is wise to work with the logarithm of the likelihood to avoid numerical overflow.
Then, we have that
\begin{align}
\log \mathcal{L}(y_{obs} | f, \alpha, M_s) = \sum_{i} -\frac{1}{2}\left(\frac{\left(y_i - s(t_i; f, \alpha)\right)^2}{\sigma^2} + \log\left(2\pi \sigma^2\right)\right)
\end{align}
We now transcribe this into `python`:

In [ ]:
def log_likelihood_Ms(time, yobs, freq, alpha, sigma=0.1):
    prediction = model_Ms(time, freq, alpha)
    res  = yobs - prediction 
    logl = -0.5 * (((res/sigma)**2) + np.log(2 * np.pi * sigma**2))
    return np.sum(logl, axis=0)

### Priors

The second part of Bayes theorem is the *prior*.
For our two-component model, we will use a simple disjoint prior (i.e. $\pi(\theta | M_s)=\pi(f| M_s)\pi(\alpha | M_s)$) with
\begin{align}
  \pi(f     | M_s) &= \textrm{Uniform}(0, 5) \\
  \pi(\alpha| M_s) &= \textrm{Uniform}(0, 1)
\end{align}
Let us create a python function to calculate the log of the prior:

In [ ]:
def log_prior_Ms(freq, alpha):
    """ Calculate the log prior under the Ms model
    
    Parameters
    ----------
    freq: array or float
        The frequency at which to calculate the prior
    alpha: array or float
        The alpha at which to calculate the prior
        
    Returns
    -------
    log_prior: array
        The log_prior calculated for all freq, alpha samples
    """
    # Convert freq, alpha to numpy arrays
    freq = np.atleast_1d(freq)
    alpha = np.atleast_1d(alpha)
    
    # Apply Uniform priors: calculate idxs of array where f, alpha in range
    f_min = 0
    f_max = 5
    f_idxs = (freq > f_min) * (freq < f_max)
    
    alpha_min = 0
    alpha_max = 1
    alpha_idxs = (alpha > alpha_min) * (alpha < alpha_max)
    
    idxs = alpha_idxs * f_idxs
    
    log_prior_volume = np.log(1/(f_max - f_min) * (1 / (alpha_max - alpha_min)))
    
    log_prior = np.zeros_like(freq)
    log_prior[idxs] = log_prior_volume
    log_prior[~idxs] = -np.inf
    return log_prior

### Parameter Estimation: Rejection Sampling

Now that we have our likelihood and prior, we will introduce **stochastic (i.e., Monte Carlo) sampling**.
We start by using the simplest type of stochastic sampling, rejection sampling.
The idea is that to draw samples from a target distribution $p(\theta | d, M_s)$ which is difficult to sample from, we first generate samples from a generating distribution $g(\theta$) which is easy to sample from and then weight the samples relative to the target distribution.
In practice you can choose any generating distribution you like, but we will use $g(\theta) = g(f)g(\alpha)$ where
\begin{align}
  g(f)      &= \textrm{Uniform}(1.8, 2.2) \\
  g(\alpha) &= \textrm{Uniform}(0.2, 0.6)
\end{align}

Our rejection sampling algorithm then proceeds as follows:

1. Draw $\theta'=[f, \alpha]$ from $g(f)$ and $g(\alpha)$
2. Calculate the probability under the target and generating distributions (i.e. $p(\theta' | d, M_s)$ and $g(\theta')$)
3. Calculate the weight $w=p(\theta' | d, M_s) / g(\theta')$
4. Draw a random number $u$ uniformly distributed in $[0, 1]$
5. If $w > u$, append $\theta'$ to a set of samples, otherwise reject it and repeat

Continue this loop until an acceptable number of samples have been drawn.
The resulting set of samples are then an approximation to $p(\theta | d, M_s)$ and be used to produce summary statistics or create plots. 

We now program the algorithm for our test data.
However, there are two important differences between this algorithm and the expression above:

1. We will work with the unnormalised distribution $p(\theta | d, M_s)$ (i.e. we don't calculate the evidence $\mathcal{Z}$).
   As a result, $w$ is also unnormalised and so it needs to be normalised before we apply step 5.
   Fortunately, we can normalize $w$ once we have a distribution of values.
2. For computational efficiency, rather than using a while loop we will instead draw a set of 100000 samples, calculate the weights for each, and then apply rejection sampling.
   This utilises numpy array optimization and also enables us to normalise the weights to a distribution.

In [ ]:
# Draw 10000 samples from g(theta)
N = 100000
freq_gsamples = np.random.uniform(1.8, 2.2, N)
alpha_gsamples = np.random.uniform(0.2, 0.6, N)

# Make time a 2D array to enable broadcasting across the samples
time_array = time[:, np.newaxis]
yobs_array = yobs[:, np.newaxis]

# Calculate the log_likelihood and log_prior for all samples
log_likelihood_vals = log_likelihood_Ms(time_array, yobs_array, freq_gsamples, alpha_gsamples)
log_prior_vals = log_prior_Ms(freq_gsamples, alpha_gsamples)
log_posterior_vals = log_likelihood_vals + log_prior_vals

# Calculate the weights
weights = np.exp(log_posterior_vals)

# Normalise the weights
weights = weights / max(weights)

# Rejection sample
keep = weights > np.random.uniform(0, 1, weights.shape)
alpha_samples = alpha_gsamples[keep]
freq_samples = freq_gsamples[keep]

The end result of this is a set of samples `freq_samples` and `alpha_samples` that approximate the posterior distribution.
We can get a quick visualisation of these by using the `corner` package:

In [ ]:
! pip install corner

In [ ]:
from corner import corner

# Create a corner plot
samples = np.array([freq_samples, alpha_samples]).T
fig = corner(samples, bins=20, labels=["f", "alpha"], show_titles=True, quantiles=[0.16, 0.5, 0.84],)

The plot above shows two 1D histograms (one for each parameter) and one 2D histogram (showing any correlations between the samples).
Areas where the posterior is large (i.e. the histogram count is high) represent the most probable values of $f$ and $\alpha$ which explain the data.

The samples can also be used to provide a summary statistic.
For example, if you wanted to report the mean and standard deviation interval for $f$, you could do something like this: 

In [ ]:
mean_f = np.mean(freq_samples)
std_f = np.std(freq_samples)
print(f"We estimate the the mean and standard deviation of frequency to be {mean_f:0.2f}+/-{std_f:0.2f}")

Typically, in GW astronomy, we use the median and a 90\% credible interval because the posterior is often non-symmetric.

In this exerise, we have learned that rejection sampling can be used to approximate the posterior distribution.
However, we should note that it is highly inefficienct.
It works okay here, because we tightly tuned the edges of $g(\theta)$, but if you go back and increase these to a wider range, you'll see the efficiency quickly drops off.
Moreover, the efficiency of rejection sampling also suffers when we start to look at problems in more than 2 parameters.

In [ ]:
# To calculate the efficiency
efficiency = len(freq_samples) / len(freq_gsamples)
print(efficiency)